In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import gutenberg
import numpy as np
from scipy.sparse import csr_matrix
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
from sklearn.feature_extraction.text import CountVectorizer
from gensim.parsing.preprocessing import remove_stopwords

In [20]:
import collections
import gensim 
from gensim.models import Word2Vec 

In [5]:
def read_file(filename):
    "Read the contents of FILENAME and return as a string."
    infile = open(filename) # windows users should use codecs.open after importing codecs
    contents = infile.read()
    infile.close()
    return contents

In [6]:
path_to_file = 'shakespeare.txt'
text = open(path_to_file, 'r').read()


In [7]:
text = remove_stopwords(text)

In [26]:
stopwords = ['mr','mrs','one','two','said','I','and','the','you','that','to','but','i','me','let','so','of','for','it','what','sir','o','a','is','as','my']

In [27]:
wordcount = {}
for word in text.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stopwords:
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1

In [29]:
n_print = int(input("How many most common words to print: "))
print("\nOK. The {} most common words are as follows\n".format(n_print))
word_counter = collections.Counter(wordcount)
for word, count in word_counter.most_common(n_print):
    print(word, ": ", count)

How many most common words to print: 20

OK. The 20 most common words are as follows

thou :  5381
thy :  4030
shall :  3569
thee :  2949
good :  2750
king :  2711
lord :  2703
come :  2409
enter :  2080
hath :  1933
love :  1883
i'll :  1739
like :  1687
man :  1653
if :  1649
know :  1605
him :  1594
this :  1585
he :  1554
by :  1535


In [14]:
tokenizer = RegexpTokenizer(r'\w+')
data = [] 
for i in sent_tokenize(text): 
    temp = [] 
    for j in tokenizer.tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [15]:
def build_corpus(text):
    porter = nltk.PorterStemmer()

    tokenizer = RegexpTokenizer(r'\w+')
    words = [] 
    for i in sent_tokenize(text): 
        for j in tokenizer.tokenize(i):
            words.append(j.lower())
    words = np.unique(words)

    porter = nltk.PorterStemmer()
    words = [porter.stem(t) for t in words]
    words = np.unique(words)
    
    word_to_index = {}
    index_to_word = {}
    counter = 0;
    for w in words:
        word_to_index[w] = counter
        index_to_word[counter] = w
        counter += 1  
    return words, word_to_index, index_to_word

In [16]:
def build_co_matrix2(text, words, word_to_index, window=1):
    porter = nltk.PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    corpus_size = len(words)
    co_matrix = np.zeros((corpus_size,corpus_size),dtype=int)
    for s in sent_tokenize(text): 
        sent = [] 
        for w in tokenizer.tokenize(s):        
            sent.append(porter.stem(w.lower()))
        for i, w in enumerate(sent):
            for j in range(max(i-window,0),min(i+window+1,len(sent))):
                co_matrix[word_to_index[w],word_to_index[sent[j]]] += 1
    return co_matrix

In [17]:
words, word_to_index, index_to_word = build_corpus(text)
matrix = build_co_matrix2(text, words, word_to_index, 5)

In [18]:
corpus_size = len(words)


In [21]:
window_list=[3,5,7,10,12,15,20,25]
for i in window_list:
    model = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = i)
    print("window size ",i)
    print("Cosine similarity between 'king' " + 
               "and 'prince' - CBOW : ", 
    model.wv.similarity('king', 'prince')) 
    

window size  3
Cosine similarity between 'king' and 'prince' - CBOW :  0.9207146
window size  5
Cosine similarity between 'king' and 'prince' - CBOW :  0.9186545
window size  7
Cosine similarity between 'king' and 'prince' - CBOW :  0.9213121
window size  10
Cosine similarity between 'king' and 'prince' - CBOW :  0.9482051
window size  12
Cosine similarity between 'king' and 'prince' - CBOW :  0.92100793
window size  15
Cosine similarity between 'king' and 'prince' - CBOW :  0.93095136
window size  20
Cosine similarity between 'king' and 'prince' - CBOW :  0.9585674
window size  25
Cosine similarity between 'king' and 'prince' - CBOW :  0.95707333


In [22]:
window_list=[3,5,7,10,12,15,20,25]
for i in window_list:
    model = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = i)
    print("window size",i)
    print("Cosine similarity between 'love' " + 
               "and 'life' - CBOW : ", 
    model.wv.similarity('love', 'life')) 
    

window size 3
Cosine similarity between 'love' and 'life' - CBOW :  0.9184166
window size 5
Cosine similarity between 'love' and 'life' - CBOW :  0.930352
window size 7
Cosine similarity between 'love' and 'life' - CBOW :  0.93428457
window size 10
Cosine similarity between 'love' and 'life' - CBOW :  0.91966665
window size 12
Cosine similarity between 'love' and 'life' - CBOW :  0.9257948
window size 15
Cosine similarity between 'love' and 'life' - CBOW :  0.92065775
window size 20
Cosine similarity between 'love' and 'life' - CBOW :  0.9252828
window size 25
Cosine similarity between 'love' and 'life' - CBOW :  0.9403358


In [23]:
model.wv.most_similar(positive='king', topn=5)

[('henry', 0.947674036026001),
 ('prince', 0.9422085285186768),
 ('duke', 0.9258636236190796),
 ('france', 0.9249615669250488),
 ('queen', 0.9104045033454895)]

In [24]:
model.wv.most_similar(positive='love', topn=5)

[('life', 0.9403358101844788),
 ('thoughts', 0.9297899007797241),
 ('honour', 0.9282357692718506),
 ('self', 0.9237463474273682),
 ('heart', 0.922876238822937)]

In [25]:
model.wv['king']

array([ 7.2539383e-01,  5.1984650e-01, -1.9127837e-01, -1.7331300e+00,
        6.3230805e-02,  2.4035861e-01, -2.7464482e-01, -4.7852714e-02,
       -2.1515999e+00, -2.9724622e-01,  1.0151055e+00,  7.2396912e-02,
       -2.4122821e-01,  9.1402426e-02, -2.5437903e-01,  9.3016815e-01,
        5.0168306e-01,  3.9127040e-01, -5.2005821e-01, -9.6255869e-01,
       -5.7866149e-02,  3.4405175e-01, -3.2040128e-01, -9.3581313e-01,
        8.4705156e-01,  1.1640203e+00,  1.8569525e+00,  1.2007906e+00,
       -1.3896519e-01, -5.4914612e-01,  7.2513305e-02, -5.6529289e-01,
        4.9120045e-01,  8.9598727e-01,  6.9172108e-01,  9.1849089e-01,
       -5.4376543e-01, -4.9530199e-01,  1.8335420e-03, -5.5325270e-01,
        8.0933225e-01,  3.7459135e-02, -6.7267589e-02,  2.7283970e-01,
        5.1148504e-01, -5.0786251e-01,  1.4372779e-01, -2.0419238e+00,
        1.3239539e+00, -4.6192834e-01, -8.0641955e-02, -4.8716417e-01,
       -1.6708932e+00, -4.0202945e-02,  3.9717317e-01, -2.4634288e-01,
      